In [1]:
import sys
sys.path.append('/opt/ml/code')

import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from my_modules.transform import get_transform
from my_modules.dataset import TrainDataset, EvalDataset
from my_modules.trainer import Trainer
from my_modules.loss import LabelSmoothing
from my_modules.inference import create_label

import torch
from torch.utils.data import DataLoader

In [2]:
df_train_path = f'df/df_labled_train.csv'
df_valid_path = f'df/df_labled_valid.csv'
df_test_path = '/opt/ml/input/data/eval/info.csv'

df_train = pd.read_csv(df_train_path)
df_valid = pd.read_csv(df_valid_path)
df_test = pd.read_csv(df_test_path)

In [3]:
def sum_label3(df):
    
    def _label(row):
        return row['age'] + 3*row['gender'] + 6*row['mask']

    df['ans'] = df.apply(_label, axis=1)
    return df[['ImageID', 'ans']]

In [4]:
device = 'cuda'

In [18]:
## add age label

transform_test = get_transform(augment=False, crop=350, resize=224, cutout=None)
test_dataset = EvalDataset(df=df_test, transform=transform_test)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=3, drop_last=False)

model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=3).to(device)
model.load_state_dict(torch.load('/opt/ml/code/save/age/extended_far_best.pt'))
df_submit = create_label(model, test_dataloader, df_test.copy(), device, target='age')

Loaded pretrained weights for efficientnet-b0


100%|██████████| 197/197 [00:24<00:00,  7.94it/s]

inference age complete!


In [19]:
df_submit.age.value_counts()

0    5334
2    3659
1    3607
Name: age, dtype: int64

In [13]:
## add mask label

transform_test = get_transform(augment=False, crop=350, resize=224, cutout=None)
test_dataset = EvalDataset(df=df_test, transform=transform_test)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=3, drop_last=False)

from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=3).to(device)
model.load_state_dict(torch.load('/opt/ml/code/save/mask/best_20_log1.pt'))
df_submit = create_label(model, test_dataloader, df_submit, device, target='mask')

Loaded pretrained weights for efficientnet-b0


100%|██████████| 197/197 [00:24<00:00,  7.95it/s]

inference mask complete!


In [14]:
## add gender label

transform_test = get_transform(augment=False, crop=350, resize=300, cutout=None)
test_dataset = EvalDataset(df=df_test, transform=transform_test)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=3, drop_last=False)

model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2).to(device)
model.load_state_dict(torch.load('/opt/ml/code/save/gender/log2.pt'))
df_submit = create_label(model, test_dataloader, df_submit, device, target='gender')

Loaded pretrained weights for efficientnet-b3


100%|██████████| 197/197 [00:30<00:00,  6.55it/s]

inference gender complete!


In [17]:
df_submit.age.value_counts()

0    5363
1    5042
2    2195
Name: age, dtype: int64

In [10]:
## sumup labels

df_submit = sum_label3(df_submit)

In [11]:
df_submit.to_csv('submit_please.csv', index=False)